In [1]:
#1.loading the json training data
import json
import pandas as pd

# Define the path to the file
file_path = r'C:\Users\DELL\OneDrive\Desktop\NEWSPJT\news\nytimes_train.json'

try:
    # Open and read the JSON file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Create a DataFrame from the JSON data
    df = pd.DataFrame(data)

    # Display the DataFrame
    display(df)

except Exception as e:
    print("An error occurred while reading the JSON file:")
    print(e)

,section,headline,article_url,article,abstract,article_id,image,caption,image_id
0,Theater,"Before 'Moonlight' and 'The Walking Dead,' a F...",https://www.nytimes.com/2017/02/21/theater/dan...,Danai Gurira and Andre Holland in a theater at...,André Holland first saw Danai Gurira at New Yo...,952a8b15-63f6-5e41-8f23-82dfbb33d1fa,https://static01.nyt.com/images/2017/02/22/art...,Danai Gurira and André Holland in a theater at...,952a8b15-63f6-5e41-8f23-82dfbb33d1fa
1,Television,What's on TV Wednesday: 'Crip Camp' and 'Dark ...,https://www.nytimes.com/2020/03/25/arts/televi...,CRIP CAMP: A DISABILITY REVOLUTION (2020) Stre...,"The documentary ""Crip Camp: A Disability Revol...",67b1b62f-43ea-59d0-bf93-a94c11845820,https://static01.nyt.com/images/2020/03/25/art...,Judy Heumann in &ldquo;Crip Camp: A Disability...,67b1b62f-43ea-59d0-bf93-a94c11845820
2,Sports,Rays Stick to Their Plan and Reach a 2nd World...,https://www.nytimes.com/2020/10/17/sports/base...,The Tampa Bay Rays told Charlie Morton it woul...,"Guided by the club's strict principles, Manage...",3da821ea-b93b-5a09-99d7-45e1711b73f0,https://static01.nyt.com/images/2020/10/19/spo...,The Rays celebrated after the final out of the...,3da821ea-b93b-5a09-99d7-45e1711b73f0
3,Art & Design,"For Robert Rauschenberg, No Artist Is an Island",https://www.nytimes.com/2017/05/11/arts/design...,"We tend to think of artists as natural loners,...","A writer travels from Brooklyn to Lafayette, L...",becf4474-d1fa-5722-b829-251530aa942b,https://static01.nyt.com/images/2017/05/14/art...,"Robert Rauschenberg performing in ""Pelican"" in...",becf4474-d1fa-5722-b829-251530aa942b
4,Theater,Jimmy Buffett's 'Margaritaville' Musical Sets ...,https://www.nytimes.com/2017/06/05/theater/jim...,Jimmy Buffett's tropical paradise will land in...,"""Escape to Margaritaville,"" which features new...",9511a127-e616-5bf5-80c6-553da4010c19,https://static01.nyt.com/images/2017/06/06/art...,"Jimmy Buffett, performing in 2016. His musical...",9511a127-e616-5bf5-80c6-553da4010c19
...,...,...,...,...,...,...,...,...,...
48983,Real Estate,Recent Commercial Real Estate Transactions,https://www.nytimes.com/2019/08/13/realestate/...,"Built in the late 1800s, this four-story prope...",Recent commercial real estate transactions in ...,edbd14e2-9f6e-588d-9b98-8c54541d2865,https://static01.nyt.com/images/2019/08/14/bus...,This building in Bushwick has two residential ...,edbd14e2-9f6e-588d-9b98-8c54541d2865
48984,Automobiles,"Volvo Recalls 31,000 Cars for Oil Pressure Sen...",https://www.nytimes.com/2013/12/31/automobiles...,"Volvo will recall nearly 31,000 S60 sedans wit...",When a mechanic forgot to put oil in a Volvo S...,11b2f98a-45a2-5109-8b4f-5b8fb1e9e2f4,https://static01.nyt.com/images/2013/12/31/aut...,The 2012 Volvo S60.,11b2f98a-45a2-5109-8b4f-5b8fb1e9e2f4
48985,Automobiles,A Family Sedan for the Speedway,https://www.nytimes.com/2014/09/14/automobiles...,Its wheelbase is 4.2 inches longer and it has ...,The 2015 Charger Hellcat will have the same su...,5c469e7a-a39b-5248-9fac-4f289c78a7cb,https://static01.nyt.com/images/2014/09/14/aut...,Dodge says Charger Hellcat can hit 204 m.p.h.,5c469e7a-a39b-5248-9fac-4f289c78a7cb
48986,Media,"Lessons From 2016 for the News Media, as the G...",https://www.nytimes.com/2016/12/25/business/ne...,Starting a weekly column about the nexus betwe...,A presidential race that made fools of pundits...,4074ae02-c26f-57cf-b32e-4e61fd334a99,https://static01.nyt.com/images/2016/12/26/bus...,President-elect Donald J. Trump after meeting ...,4074ae02-c26f-57cf-b32e-4e61fd334a99


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, Concatenate
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import text


# Load the pre-trained ResNet50 model (excluding the classification layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features from images using the ResNet50 model
def extract_image_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = base_model.predict(img_array)
    features = features.reshape((-1,))
    return features

# Preprocess text data
max_seq_length = 100  # Maximum sequence length for padding
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['article'])
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(df['article'])
X_text = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

# Prepare image features and text sequences
image_features = np.array([extract_image_features(img_path) for img_path in df['image_url']])
text_sequences = np.array(X_text)

# Define the captioning model
input_image = Input(shape=(image_features.shape[1],))
input_text = Input(shape=(max_seq_length,))
image_embedding = Dense(256, activation='relu')(input_image)
text_embedding = Embedding(vocab_size, 256, mask_zero=True)(input_text)
text_lstm = LSTM(256)(text_embedding)
combined = Concatenate()([image_embedding, text_lstm])
output = Dense(vocab_size, activation='softmax')(combined)

captioning_model = Model(inputs=[input_image, input_text], outputs=output)
captioning_model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the captioning model
captioning_model.fit([image_features, text_sequences], text_sequences, epochs=10, batch_size=32)

# Generate captions for each news article's image
captions = []
for i in range(len(df)):
    image_feature = np.expand_dims(image_features[i], axis=0)
    text_sequence = np.expand_dims(text_sequences[i], axis=0)
    caption = captioning_model.predict([image_feature, text_sequence])
    captions.append(caption)

# Add captions to DataFrame
df['caption'] = captions

# Save DataFrame with captions
df.to_csv('news_articles_with_captions.csv', index=False)

NameError: name 'preprocessing' is not defined

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, Concatenate
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import text


# Load the pre-trained ResNet50 model (excluding the classification layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features from images using the ResNet50 model
def extract_image_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = base_model.predict(img_array)
    features = features.reshape((-1,))
    return features

# Preprocess text data
max_seq_length = 100  # Maximum sequence length for padding
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(df['article'])
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(df['article'])
X_text = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

# Prepare image features and text sequences
image_features = np.array([extract_image_features(img_path) for img_path in df['image']])
text_sequences = np.array(X_text)

# Define the captioning model
input_image = Input(shape=(image_features.shape[1],))
input_text = Input(shape=(max_seq_length,))
image_embedding = Dense(256, activation='relu')(input_image)
text_embedding = Embedding(vocab_size, 256, mask_zero=True)(input_text)
text_lstm = LSTM(256)(text_embedding)
combined = Concatenate()([image_embedding, text_lstm])
output = Dense(vocab_size, activation='softmax')(combined)

captioning_model = Model(inputs=[input_image, input_text], outputs=output)
captioning_model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the captioning model
captioning_model.fit([image_features, text_sequences], text_sequences, epochs=10, batch_size=32)

# Generate captions for each news article's image
captions = []
for i in range(len(df)):
    image_feature = np.expand_dims(image_features[i], axis=0)
    text_sequence = np.expand_dims(text_sequences[i], axis=0)
    caption = captioning_model.predict([image_feature, text_sequence])
    captions.append(caption)

# Add captions to DataFrame
df['caption'] = captions

# Save DataFrame with captions
df.to_csv('news_articles_with_captions.csv', index=False)

OSError: [Errno 22] Invalid argument: 'https://static01.nyt.com/images/2017/02/22/arts/22WHENWEMET1/22WHENWEMET1-facebookJumbo-v2.jpg?year=2017&h=550&w=1050&s=c23807ec313c0fc286e59ef047d8ec2ffb842fd4368b350e0be6c4895ec1a0ba&k=ZQJBKqZ0VN'